In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import glob

In [ ]:
# read data from the download service -- alert-based

dirname = "/media3/CRP7/hosts/data/TNS/Apr2023/ftransfer_ztf_2023-08-30_761838/"
data_all = pd.read_parquet(dirname)

In [ ]:
# choose object id
name = 'ZTF19adczgih'

In [ ]:
# this will take a long time
flist = glob.glob('/media3/CRP7/hosts/data/TNS/Apr2023/obj_info/*.pickle')

data_list = []
for fname in flist:
    data_temp = pd.read_pickle(fname)
    data_list.append(data_temp)

data_pd = pd.concat(data_list, ignore_index=True)

In [ ]:
list(data_pd.keys())

In [ ]:
data_pd.shape

In [ ]:
x = data_pd[data_pd['i:objectId'].values == name]['i:jd'].values
y = data_pd[data_pd['i:objectId'].values == name]['i:magpsf'].values
yerr = data_pd[data_pd['i:objectId'].values == name]['i:sigmapsf'].values
fid = data_pd[data_pd['i:objectId'].values == name]['i:fid'].values

In [ ]:
plt.figure(figsize=[16,5])

for i in range(1,3):
    plt.subplot(1,2, i)
    flag = fid == i
    plt.errorbar(x[flag], y[flag], yerr=yerr[flag], fmt='o')
    plt.xlabel('JD')
    plt.ylabel('magpsf')

plt.show()

In [ ]:
columns = [
    'b:cutoutScience_stampData',
    'b:cutoutTemplate_stampData',
    'b:cutoutDifference_stampData'
]

stamps = data_pd[data_pd['i:objectId'].values == name]

In [ ]:
np.unique(stamps['d:mangrove_2MASS_name'].values, return_counts=True)

In [ ]:
flag_stamps = np.array([str(stamps['d:mangrove_2MASS_name'].values[i]) for i in range(stamps.shape[0])]) == 'nan'

In [ ]:
np.unique(flag_stamps, return_counts=True)

In [ ]:
title = ['Science', 'Template', 'Difference']

# select a random alert
for indx in np.arange(stamps.shape[0])[~flag_stamps]:
    
    fig, axes = plt.subplots(1,3, figsize=(15,5))
    objid = stamps.iloc[indx]['i:objectId']
    fwhm = str(round(stamps.iloc[indx]['i:fwhm'], 3))
    aimage = str(round(stamps.iloc[indx]['i:aimage'], 3))
    bimage = str(round(stamps.iloc[indx]['i:bimage'], 3))
    isdiffpos = str(bool(stamps.iloc[indx]['i:isdiffpos']))
    classname = str(stamps.iloc[indx]['v:classification'])
    
    fig.suptitle('indx: ' + str(indx) + '  -  ' + objid + ' ; ' + classname + '\n  fwhm = ' + fwhm + ', ' + \
                'aimage: ' + aimage + ',  bimage: ' + bimage + ', isdiffpos: ' + isdiffpos)

    for i,ax in enumerate(axes.flat):
        ax.set_title(title[i])
        data = np.array(stamps[columns[i]].values[indx])

        ax.imshow(data.astype(float))

    plt.show()

In [ ]:
# plot positions
ra = stamps['i:ra'].values
dec = stamps['i:dec'].values


plt.figure(figsize=(8,5))
plt.scatter(ra[flag_stamps], dec[flag_stamps], label='nan')
plt.scatter(ra[~flag_stamps], dec[~flag_stamps], label='number')

plt.xlabel('ra')
plt.ylabel('dec')
plt.legend()
plt.show()

